In [ ]:
import json
import openai
import os
import panel as pn

openai.api_key = 'MY_OPENAI_API_KEY' # replace with your OpenAI API key

In [ ]:
JSON_STORE_PATH = './jsons' # path of the JSON file with the DB schema
JSON_NAME = 'my_schema.json' # name of the JSON file
DB = 'MY_DB' # type of the DB where the schema is stored (for ChatGPT to generate queries for a specific SQL dialect)
print(os.listdir(JSON_STORE_PATH))

In [ ]:
def get_completion(prompt, model='gpt-3.5-turbo', temperature=0):
    messages = [{'role': 'user', 'content': prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )
    return response.choices[0].message['content']

def get_completion_from_messages(messages, model='gpt-3.5-turbo', temperature=0):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )
    return response.choices[0].message['content']

In [ ]:
def collect_messages(_):
    if inp.value == '':
        panels.append(pn.Row('System:', pn.pane.Markdown('The input is empty.', width=610, style={'background-color': '#FFB1B1'})))
    else:
        prompt = inp.value_input
        inp.value = ''
        context.append({'role': 'user', 'content': f'{prompt}'})
        response = get_completion_from_messages(context) 
        context.append({'role': 'assistant', 'content': f'{response}'})
        panels.append(pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
        panels.append(pn.Row('Assistant:', pn.pane.Markdown(response, width=600, style={'background-color': '#F6F6F6'})))
    return pn.Column(*panels)

In [ ]:
# read the DB schema from the JSON file
with open(f'{JSON_STORE_PATH}/{JSON_NAME}') as f:
    schema = json.load(f)

In [ ]:
# define the context for ChatGPT providing the DB schema
# if you want to reset the chat history, rerun this and the next cell
context = [{'role': 'system', 'content': f'''
You are an automated service that helps analyze data. \
You answer business questions using SQL queries within the provided database. \
You provide a description for each generated SQL query. \
The database schema is presented in JSON format below, delimited by triple backticks. \
The schema is stored in the {DB} database. \
The user can request code generation for a specific programming language \
to visualize the results obtained from executing the SQL query you provided. \
You respond in a short formal style.
The database schema:
```{schema}```
'''}]

In [ ]:
# UI - chat between the user and ChatGPT

pn.extension()

panels = [] # collect display 

inp = pn.widgets.TextInput(value='Hi', value_input='Hi', placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name='Chat!')

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=400),
)

dashboard